In [1]:
#main chatbot script

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
import os
import openai

os.environ["OPENAI_API_KEY"] = ""


In [8]:
from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage
from llama_index.readers.file import PDFReader
import os

def load_and_combine_pdfs(pdf_paths):
    reader = PDFReader()
    documents = []
    for path in pdf_paths:
        documents.extend(reader.load_data(file=path))
    return documents

def get_index(data, index_name):
    if not os.path.exists(index_name):
        print("Building index:", index_name)
        index = VectorStoreIndex.from_documents(data, show_progress=True)
        index.storage_context.persist(persist_dir=index_name)
    else:
        index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=index_name)
        )
    return index

# List of PDF files to be indexed
pdf_paths = ["nutrition_info/ting.pdf", "nutrition_info/performance.pdf", "nutrition_info/fat_loss.pdf", "nutrition_info/overall_nutrition.pdf"]  # Adding more docs to train from
combined_data = load_and_combine_pdfs(pdf_paths)
combined_index = get_index(combined_data, "combined_deficiency")
combined_engine = combined_index.as_query_engine()

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [9]:
from dotenv import load_dotenv
load_dotenv()

import os
import pandas as pd
from llama_index.core.query_engine import PandasQueryEngine
#from note_engine import note_engine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

# Detailed instructions for the usage of the generated Python code with pandas
instruction_str = """
1. Convert the query to executable Python code using Pandas.
2. The final line of code should be a Python expression that can be called with the `eval()` function.
3. Ensure the code represents a solution to the query regarding nutrition data analysis.
4. PRINT ONLY THE EXPRESSION, ensuring it is syntactically correct and does not include extraneous quotes.
5. Focus on accuracy and relevance to the query, pulling directly from nutritional data sets.
"""

# Template for prompting which includes dynamic data display and tailored instructions
new_prompt = PromptTemplate(
    """\
    You are working with a pandas dataframe named `df` that includes nutritional data similar to the CDC's Second Nutrition Report.
    Here is a preview of the dataframe (`df.head()`):
    {df_str}

    Please follow these detailed instructions:
    {instruction_str}
    Query: {query_str}

    Write the Python expression that answers the query:
    """
)

# Context description to inform the agent about its operational purpose
context = """Purpose: Assist users by providing accurate information and analysis based on nutrition data.
            This agent aids in interpreting population statistics and specifics about nutrient deficiencies
            and health outcomes from structured data sources like the CDC's NHANES nutrition reports."""



# json_file_paths = [
#     os.path.join("data", "foundation.json"),
#     os.path.join("data", "legacy.json"),
#     os.path.join("data", "survey.json")
# ]

# dataframes = [pd.read_json(path) for path in json_file_paths]
# combined_df = pd.concat(dataframes, ignore_index=True)

# population_query_engine = PandasQueryEngine(
#     df=combined_df, verbose=True, instruction_str=instruction_str
# )
#population_query_engine.update_prompts({"pandas_prompt": new_prompt})

tools = [
    QueryEngineTool(
        query_engine=combined_engine,
        metadata=ToolMetadata(
            name="nutri_data",
            description="this gives detailed information about common nutritional deficiencies",
        ),
    ),
]

llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context)

while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    result = agent.query(prompt)
    print(result)

ReActChatFormatter.from_context is deprecated, please use `from_defaults` instead.


Enter a prompt (q to quit):  give me tips on constructing a diet for fat loss


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The user is asking for tips on constructing a diet for fat loss. I can provide general guidelines based on nutrition data.
Action: nutri_data
Action Input: {'input': 'fat loss'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Observation: Research has been conducted on the effectiveness of different dietary approaches for weight loss, including low-fat and low-carbohydrate diets. Studies have explored the impact of meal replacements, dietary fats, and various macronutrient compositions on weight management. Additionally, there is a 

Enter a prompt (q to quit):  give me some meal ideas for a caloric deficit


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The user is asking for meal ideas for a caloric deficit. I can provide some general suggestions based on nutritional guidelines.
Action: nutri_data
Action Input: {'input': 'caloric deficit meal ideas'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Observation: Meal ideas for creating a caloric deficit could include options like a salad with mixed greens, grilled chicken, and a vinaigrette dressing, a vegetable stir-fry with tofu or lean protein, a quinoa bowl with roasted vegetables and chickpeas, or a turkey and avocado wrap with

Enter a prompt (q to quit):  q
